In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [20]:
import pandas as pd
import numpy as np
import pickle
import joblib
from envs.film_env import FilmRecEnv
import random 



## Loading

In [21]:
catalog_df = pd.read_pickle("data/catalog_df.pkl")            # load your DataFrame
with open("data/audience_models/rf_model.pkl", "rb") as f:    # load your frozen viewership model
    audience_model = pickle.load(f)
q_model = joblib.load("models/q_model.joblib")

In [ ]:
env = FilmRecEnv(
    catalog_df     = catalog_df,
    audience_model = audience_model,
    memory_size    = 5,
    rec_list_size  = 5,
)

## Context and Recommendation

In [35]:
context = {
        "day": 'wed',
        "time":   'night',
        "season": 'Spring'
    }

In [36]:
env.state = env.reset(context)

# 2) Compute Q‐values and mask memory
q_vals = q_model.predict(env.state)            # shape (n_films,)
if env.memory:
    q_vals[env.memory] = -np.inf

# top‐K by Q‐value
idx = np.argpartition(q_vals, -env.K)[-env.K:] # get indices from top-K but not sorted, more efficient
slate = idx[np.argsort(-q_vals[idx])] # sort the top-K indices by Q-value
slate_names = [catalog_df.iloc[i]['title'] for i in slate]
catalog_df.iloc[slate]

,title,adult,duration,genres,original_language,popularity,release_date,revenue,tmdb_id,vote_average,missing_release_date,missing_tmdb,is_movie
150,Le crime lui va si bien,False,94.083333,[],unknown,0.0000,1900-01-01,0.0,NaN,0.0,True,True,True
347,Quatre mariages et un coup de foudre,False,87.933333,[],unknown,0.0000,1900-01-01,0.0,NaN,0.0,True,True,True
436,Body Cam,False,89.333333,"[{'id': 99, 'name': 'Documentaire'}, {'id': 80...",en,0.1546,1900-01-01,0.0,1013147.0,0.0,True,False,True
